In [1]:
import pyslammer as slam
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
histories = slam.sample_ground_motions()
ky = 0.15
motion = histories["Northridge_VSP-360"]
dt = motion[0][1] - motion[0][0]


In [3]:
sorted_histories = sorted(histories.keys())

In [4]:
%%time
for name in sorted_histories:
    motion = histories[name]
    dt = motion[0][1] - motion[0][0]
    da = slam.Decoupled()
    da.run_sliding_analysis(ain_p=motion[1]/9.80665,
                                  uwgt_p=20.0,
                                  height_p=50.0,
                                  vs_p=600.0,
                                  damp1_p=.05,
                                  refstrain_p=0.05,
                                  dt_p=dt,
                                  scal_p=1.0,
                                  g_p=9.80665,
                                  vr_p=600.0,
                                  ca=[[1],[ky]],
                                  dv3_p=True)
    pga = max(abs(motion[1]))/9.80665
    print(f"{name}: PGA = {pga:.2f}, decoupled disp = {da.s[-1]*100:.2f}cm")
    # ax.plot(motion[0],da.s*100,label=name)
# ax.legend()

Cape_Mendocino_1992_PET-090: PGA = 0.21, decoupled disp = 0.01cm
Chi-Chi_1999_TCU068-090: PGA = 0.57, decoupled disp = 72.94cm
Coalinga_1983_PVB-045: PGA = 0.38, decoupled disp = 7.77cm
Duzce_1999_375-090: PGA = 0.51, decoupled disp = 2.69cm
Imperial_Valley_1979_BCR-230: PGA = 0.77, decoupled disp = 33.03cm
Kobe_1995_TAK-090: PGA = 0.62, decoupled disp = 114.90cm
Kocaeli_1999_ATS-090: PGA = 0.18, decoupled disp = 0.03cm
Landers_1992_LCN-345: PGA = 0.79, decoupled disp = 8.15cm
Loma_Prieta_1989_HSP-000: PGA = 0.37, decoupled disp = 10.19cm
Mammoth_Lakes-1_1980_CVK-090: PGA = 0.42, decoupled disp = 2.71cm
Mammoth_Lakes-2_1980_CVK-090: PGA = 0.27, decoupled disp = 0.38cm
Morgan_Hill_1984_CYC-285: PGA = 1.30, decoupled disp = 22.53cm
N_Palm_Springs_1986_WWT-180: PGA = 0.49, decoupled disp = 7.53cm
Nahanni_1985_NS1-280: PGA = 1.10, decoupled disp = 8.07cm
Nisqually_2001_UNR-058: PGA = 0.27, decoupled disp = 0.87cm
Northridge_VSP-360: PGA = 0.93, decoupled disp = 29.75cm
CPU times: user 1.07

In [11]:
# plt.close("all")
# # plt.plot(motion[0],motion[1])
# # plt.plot(motion[0],da.udotdot)
# # plt.plot(motion[0],da.avgacc)
# # # plt.plot(motion[0],da.sdot)
# plt.plot(motion[0],da.s)
# plt.title(f"Total displacement: {da.s[-1]*100:.1f}cm")
# 
# plt.show()

In [6]:
da.damp1

In [7]:
class Point:
    def __init__(self, x, y):
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

In [8]:
p = Point(1,2)

In [9]:
p._x